In [1]:
import numpy as np
import tensorflow as tf
import shutil

In [2]:
n_features = 4
m_examples = 47

We have $X$, $m$ examples. Each example (each row) has $n$ features. Thus we have:
$$ 
\left[\begin{array}{cccc}
x^{(1)}_{1} & x^{(1)}_{2} & \ldots & x^{(1)}_{n} \\
x^{(2)}_{1} & x^{(2)}_{2} & \ldots & x^{(2)}_{n} \\
& & \vdots \\
x^{(m)}_{1} & x^{(m)}_{2} & \ldots & x^{(m)}_{n}
\end{array}\right] \times
\left[\begin{array}{c}
w_1 \\ w_2 \\ \vdots \\ w_n
\end{array}\right] + b =
\left[\begin{array}{c}
y_1 \\ y_2 \\ \vdots \\ v_n
\end{array}\right]
$$

In [3]:
# The input examples. The i-th row contains the i-th example. 
X_true = np.random.rand(m_examples, n_features)

# This is what we wish linear regression to learn.
w_true = np.random.rand(n_features, 1) * 100.0
# The constant
b_true = np.random.rand(1) * 100.0 - 50.0

print "w = [%s]" % ', '.join(['%.2f' % x for x in w_true])
print "b = %.2f" % b_true[0]

# Noise
noise = np.random.rand(m_examples, 1) / 100.0

# These are our labeled "results". 
y_true = (np.matmul(X_true, w_true) + b_true) + noise

w = [5.48, 90.52, 48.29, 38.47]
b = -9.33


In [4]:
# Placeholder that is fed input data.
X_in = tf.placeholder(tf.float32, [None, n_features], "X_in")

# The model: we assume y = X_in * w + b
w = tf.Variable(tf.random_normal((n_features, 1)), name="w")
b = tf.Variable(tf.constant(0.1, shape=[]), name="b")
h = tf.add(tf.matmul(X_in, w), b, name="h")

# Placeholder that is fed observed results.
y_in = tf.placeholder(tf.float32, [None, 1], "y_in")

# The loss function: we are minimizing square root of mean 
loss_op = tf.reduce_mean(tf.square(tf.subtract(y_in, h)), name="loss")
train_op = tf.train.GradientDescentOptimizer(0.3).minimize(loss_op)

# The total number of batches.
batch_count = 200

# Steps per batch
steps_per_batch = 5

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    # Clear any existing logs
    shutil.rmtree("/tmp/linreg_log", True)
    writer = tf.summary.FileWriter("/tmp/linreg_log", 
                                   sess.graph)
    loss_summ = tf.summary.scalar("loss", loss_op)
    for batch in range(batch_count):
        for step in range(steps_per_batch):
            sess.run(train_op, 
                     feed_dict={X_in: X_true, y_in: y_true})
        writer.add_summary(
          sess.run(loss_summ, 
                   feed_dict={X_in: X_true, y_in: y_true}), 
          batch * steps_per_batch)
    w_computed = sess.run(w)
    b_computed = sess.run(b)

In [5]:
print "w computed [%s]" % ', '.join(['%.5f' % x for x in w_computed.flatten()])
print "w actual   [%s]" % ', '.join(['%.5f' % x for x in w_true.flatten()])
print "b computed %.3f" % b_computed
print "b actual  %.3f" % b_true[0]

w computed [5.48375, 90.52216, 48.28834, 38.46674]
w actual   [5.48446, 90.52165, 48.28952, 38.46534]
b computed -9.326
b actual  -9.331
